In [1]:
# general script 

import json
from collections import defaultdict
from scipy.optimize import linear_sum_assignment
import numpy as np

# Load ground truth and DeepSORT JSON files
with open('groundtruth.json', 'r') as f:
    gt_data = json.load(f)
with open('hypo.json', 'r') as f:
    ds_data = json.load(f)

# Initialize variables for calculating MOTA and MOTP metrics
num_matches = 0
num_misses = 0
num_false_positives = 0
num_mismatches = 0
total_distance = 0

# Iterate through each frame of data
for frame in range(len(gt_data[0]['frames'])):
    # Get ground truth and DeepSORT data for the current frame
    gt_frame = gt_data[0]['frames'][frame]
    ds_frame = ds_data[0]['frames'][frame]

    # Get the ground truth and DeepSORT bounding boxes for the current frame
    
    gt_boxes = [[d['x'], d['y'], d['width'], d['height']] for d in gt_frame['annotations']]
    ds_boxes = [[d['x'], d['y'], d['width'], d['height']] for d in ds_frame['hypotheses']]

    # Calculate the intersection over union (IOU) between ground truth and DeepSORT bounding boxes
    iou_matrix = []
    for gt_box in gt_boxes:
        iou_row = []
        for ds_box in ds_boxes:
            x1, y1, w1, h1 = gt_box
            x2, y2, w2, h2 = ds_box
            left = max(x1, x2)
            top = max(y1, y2)
            right = min(x1 + w1, x2 + w2)
            bottom = min(y1 + h1, y2 + h2)
            if right < left or bottom < top:
                iou_row.append(0.0)
                continue
            intersection_area = (right - left) * (bottom - top)
            
            #bounding box area
            bb1_area = w1 * h1
            bb2_area = w2 * h2
            
            iou_row.append(intersection_area / float(bb1_area + bb2_area - intersection_area))
            
        iou_matrix.append(iou_row)

    # Find matches between ground truth and DeepSORT bounding boxes using the Hungarian algorithm
    if len(gt_boxes) >0 and len(ds_boxes) > 0:
        
        
        row_ind, col_ind = linear_sum_assignment(1 - np.array(iou_matrix))
        matches = [(row_ind[i], col_ind[i]) for i in range(len(row_ind)) if iou_matrix[row_ind[i]][col_ind[i]] > 0.5]
    else:
        matches= []
    # Update MOTA and MOTP metrics with the current frame data
    num_matches += len(matches)
    num_misses += len(gt_boxes) - len(matches)
    num_false_positives += len(ds_boxes) - len(matches)
    total_distance += sum([iou_matrix[i][j] for i, j in matches])


# Calculate the final MOTA and MOTP values
mota = 1 - (num_misses + num_false_positives + num_mismatches) / sum([len(frame['annotations']) for frame in gt_data[0]['frames']])
motp = 1 - (total_distance / num_matches)

print(f'MOTA: {mota}')
print(f'MOTP: {motp}')


MOTA: 0.21974522292993626
MOTP: 0.27530147356762136
